# Experiments

In [1]:
import pickle 
import pandas as pd
from dice_ml.utils import helpers
from prompts import *
from utils import *
from exp_machines import *

In [2]:
from pathlib import Path
directory = Path("./experiments")
# Create the directory if it does not exist
directory.mkdir(parents=True, exist_ok=True)

In [3]:
# Load the model we want to explain
with open("""./models/loan_model.pkl""", 'rb') as file:
    model = pickle.load(file)

#Load the train and test data set
train_dataset = pd.read_csv('./data/adult_train_dataset.csv')
test_dataset = pd.read_csv('./data/adult_test_dataset.csv')

#Load the examples we will try to explain
test_df = pd.read_csv('./data/test_examples.csv')
test_df = test_df.drop(columns=['income'], axis = 1)

In [19]:
#create datasets to test
test100 = test_df.iloc[-100:].reset_index(drop = True)
#test100.to_csv('/data/test100.csv', index = False)

In [2]:
from pathlib import Path
directory = Path("./experiments/zero_shot")
# Create the directory if it does not exist
directory.mkdir(parents=True, exist_ok=True)

In [10]:
test100 = pd.read_csv('test100.csv')
model_description = """ML-system that predicts wether a person will earn more than 50k $ a year"""
prompt = 'zero'
for prompt in ['zero', 'one', 'ToT']:
    for cf in [1,3,5]:
        if prompt == 'ToT':
            exp_m = ToTLLMExplanation4CFs(model = model, #Load the model we want to explain
                                    model_description = """ML-system that predicts wether a person will earn more than 50k $ a year""", # brief explanation of the ML model
                                    backend='sklearn', # Framework used to build the model (used to generate counterfactuals)
                                    dataset_info=string_info(train_dataset.columns, helpers.get_adult_data_info()) , # string information about the dataset
                                    continuous_features=['age', 'hours_per_week'], # Necessary for the counterfactual generation
                                    outcome_name= 'income', #Necessary for counterfactual generation
                                    training_set=train_dataset, #Necessary for counterfactual generation
                                    test_set= test_dataset, #Necessary to  check novelty of the evaluation example
                                    llm='gpt-4o', #LLM used, works with Langchain
                                    prompt_type=prompt, # zero or one
                                    n_counterfactuals=cf, #Number of counterfactuals used in the explanation 
                                    user_input=False, #Human in the loop helping select the causes
                                    branches = 3
                                )
        else:
            exp_m = LLMExplanation4CFs(model = model, #Load the model we want to explain
                                    model_description = """ML-system that predicts wether a person will earn more than 50k $ a year""", # brief explanation of the ML model
                                    backend='sklearn', # Framework used to build the model (used to generate counterfactuals)
                                    dataset_info=string_info(train_dataset.columns, helpers.get_adult_data_info()) , # string information about the dataset
                                    continuous_features=['age', 'hours_per_week'], # Necessary for the counterfactual generation
                                    outcome_name= 'income', #Necessary for counterfactual generation
                                    training_set=train_dataset, #Necessary for counterfactual generation
                                    test_set= test_dataset, #Necessary to  check novelty of the evaluation example
                                    llm='gpt-4o', #LLM used, works with Langchain
                                    prompt_type=prompt, # zero or one
                                    n_counterfactuals=cf, #Number of counterfactuals used in the explanation 
                                    user_input=False #Human in the loop helping select the causes
                                )



        exp_m.fit()
        for i in range(test100.shape[0]):
            try:
                example_label, n_rules, rules_followed, first_rule, second_rule,third_rule,in_cfs, in_dataset = exp_m.explain_evaluate(example = test.iloc[[i]], verbose = False)
                os.rename('./temp_files/temp_csv.csv', f'./experiments/ex_{cf}cfs_{i}.csv')
                os.rename('evaluation.csv', f'eval_{cf}cfs_{i}.csv')            
                test100.loc[i, str(cf) + '_cfs_' + prompt + '_label'] = example_label
                test100.loc[i, str(cf) + '_cfs_' +prompt + '_rules'] = n_rules
                test100.loc[i, str(cf) + '_cfs_' +prompt + '_rules_followed'] = rules_followed
                test100.loc[i, str(cf) + '_cfs_' + +prompt + '_rule_1'] = first_rule
                test100.loc[i, str(cf) + '_cfs_' + +prompt + '_rule_2'] = second_rule
                test100.loc[i, str(cf) + '_cfs_' + +prompt + '_rule_3'] = third_rule
                test100.loc[i, str(cf) + '_cfs_' +prompt + '_in_cfs'] = in_cfs
                test100.loc[i, str(cf) + '_cfs_' +prompt + '_in_data'] = in_dataset
                test100.loc[i, str(cf) + '_cfs_' +prompt + '_status'] = 0
            except Exception as e:
                test100.loc[i, str(cf) + '_cfs_' +prompt + '_status'] = 1
test100

one


100%|██████████| 1/1 [00:00<00:00,  1.65it/s]


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,one_label,one_rules,one_rules_followed,one_rule_1,one_rule_2,one_rule_3,one_in_cfs,one_in_data,one_status
0,45,Self-Employed,HS-grad,Married,Blue-Collar,White,Male,50,1.0,3.0,3.0,1.0,1.0,1.0,False,False,0.0
1,46,Private,HS-grad,Divorced,Blue-Collar,White,Male,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,49,Private,HS-grad,Widowed,Service,White,Male,40,1.0,3.0,3.0,1.0,1.0,1.0,False,True,0.0
3,26,Government,Bachelors,Single,Professional,Other,Female,40,1.0,3.0,2.0,1.0,1.0,0.0,False,False,0.0
4,45,Government,Bachelors,Divorced,Service,White,Female,40,0.0,4.0,4.0,1.0,1.0,1.0,False,False,0.0
5,35,Private,Some-college,Married,Blue-Collar,White,Male,60,1.0,3.0,3.0,1.0,1.0,1.0,False,True,0.0
6,29,Private,HS-grad,Single,Blue-Collar,Other,Male,40,1.0,3.0,3.0,1.0,1.0,1.0,False,False,0.0
7,40,Self-Employed,Some-college,Married,Service,White,Male,50,0.0,3.0,2.0,1.0,1.0,0.0,False,False,0.0
8,28,Other/Unknown,HS-grad,Married,Other/Unknown,White,Female,20,0.0,3.0,2.0,1.0,1.0,0.0,False,False,0.0
9,20,Private,Some-college,Single,Service,White,Female,40,1.0,3.0,3.0,1.0,1.0,1.0,False,False,0.0


In [ ]:
exp_m = ToTLLMExplanation4CFs( model, backend, model_description, dataset_info, continuous_features, outcome_name,training_set, test_set, llm = 'gpt-4o', prompt_type = 'zero', n_counterfactuals=5, branches = 3, user_input = False)

In [11]:
def get_metrics(df,prompt):
    validity = df[f'{prompt}_label'].mean()
    rules = df[f'{prompt}_rules'].mean()
    rules_ratio = np.mean(df[f'{prompt}_rules_followed']/df[f'{prompt}_rules'])
    first = df[f'{prompt}_rule_1'].mean()
    second = df[f'{prompt}_rule_2'].mean()
    third = df[f'{prompt}_rule_3'].mean()
    return [validity, rules, rules_ratio, first, second, third]
get_metrics(test10,'one')

[0.6666666666666666,
 3.111111111111111,
 0.8888888888888888,
 1.0,
 1.0,
 0.6666666666666666]

In [8]:
test10 = pd.read_csv('test10.csv')
model_description = """ML-system that predicts wether a person will earn more than 50k $ a year"""
prompts = ['one']
for prompt in prompts:
    print(prompt)
    exp_m = ExplanationMachine2(model, model_description, string_info(dataset.columns,adult_info), train_dataset, test_dataset,prompt,1 )
    exp_m.fit()
    for i in range(test10.shape[0]):
        try:
            example_label, n_rules, rules_followed, first_rule, second_rule,third_rule,in_cfs, in_dataset = exp_m.explain_evaluate(example = test.iloc[[i]], verbose = False)
            os.rename('temp_csv.csv', f'ex_1fcs_{i}.csv')
            os.rename('evaluation.csv', f'eval_1fcs_{i}.csv')
            test10.loc[i, prompt + '_label'] = example_label
            test10.loc[i, prompt + '_rules'] = n_rules
            test10.loc[i, prompt + '_rules_followed'] = rules_followed
            test10.loc[i, prompt + '_rule_1'] = first_rule
            test10.loc[i, prompt + '_rule_2'] = second_rule
            test10.loc[i, prompt + '_rule_3'] = third_rule
            test10.loc[i, prompt + '_in_cfs'] = in_cfs
            test10.loc[i, prompt + '_in_data'] = in_dataset
            test10.loc[i, prompt + '_status'] = 0
        except Exception as e:
            test10.loc[i, prompt + '_status'] = 1
test10

one


100%|██████████| 1/1 [00:00<00:00,  2.91it/s]


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,one_label,one_rules,one_rules_followed,one_rule_1,one_rule_2,one_rule_3,one_in_cfs,one_in_data,one_status
0,45,Self-Employed,HS-grad,Married,Blue-Collar,White,Male,50,1.0,3.0,2.0,1.0,1.0,0.0,False,False,0.0
1,46,Private,HS-grad,Divorced,Blue-Collar,White,Male,30,1.0,3.0,3.0,1.0,1.0,1.0,False,True,0.0
2,49,Private,HS-grad,Widowed,Service,White,Male,40,1.0,3.0,3.0,1.0,1.0,1.0,False,False,0.0
3,26,Government,Bachelors,Single,Professional,Other,Female,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,45,Government,Bachelors,Divorced,Service,White,Female,40,0.0,3.0,3.0,1.0,1.0,1.0,False,False,0.0
5,35,Private,Some-college,Married,Blue-Collar,White,Male,60,1.0,3.0,3.0,1.0,1.0,1.0,False,True,0.0
6,29,Private,HS-grad,Single,Blue-Collar,Other,Male,40,0.0,3.0,3.0,1.0,1.0,1.0,False,False,0.0
7,40,Self-Employed,Some-college,Married,Service,White,Male,50,1.0,3.0,1.0,1.0,0.0,0.0,False,False,0.0
8,28,Other/Unknown,HS-grad,Married,Other/Unknown,White,Female,20,1.0,3.0,2.0,1.0,1.0,0.0,True,False,0.0
9,20,Private,Some-college,Single,Service,White,Female,40,1.0,3.0,3.0,1.0,1.0,1.0,False,False,0.0


In [9]:
def get_metrics(df,prompt):
    validity = df[f'{prompt}_label'].mean()
    rules = df[f'{prompt}_rules'].mean()
    rules_ratio = np.mean(df[f'{prompt}_rules_followed']/df[f'{prompt}_rules'])
    first = df[f'{prompt}_rule_1'].mean()
    second = df[f'{prompt}_rule_2'].mean()
    third = df[f'{prompt}_rule_3'].mean()
    return [validity, rules, rules_ratio, first, second, third]
get_metrics(test10,'one')

[0.7777777777777778,
 3.0,
 0.8518518518518519,
 1.0,
 0.8888888888888888,
 0.6666666666666666]

In [9]:
test10 = pd.read_csv('test10.csv')
model_description = """ML-system that predicts wether a person will earn more than 50k $ a year"""
prompts = ['one']
for prompt in prompts:
    print(prompt)
    exp_m = ExplanationMachine2(model, model_description, string_info(dataset.columns,adult_info), train_dataset, test_dataset,prompt,3 )
    exp_m.fit()
    for i in range(test10.shape[0]):
        try:
            example_label, n_rules, rules_followed, first_rule, second_rule,third_rule,in_cfs, in_dataset = exp_m.explain_evaluate(example = test.iloc[[i]], verbose = False)
            os.rename('temp_csv.csv', f'ex_3fcs_{i}.csv')
            os.rename('evaluation.csv', f'eval_3fcs_{i}.csv')
            test10.loc[i, prompt + '_label'] = example_label
            test10.loc[i, prompt + '_rules'] = n_rules
            test10.loc[i, prompt + '_rules_followed'] = rules_followed
            test10.loc[i, prompt + '_rule_1'] = first_rule
            test10.loc[i, prompt + '_rule_2'] = second_rule
            test10.loc[i, prompt + '_rule_3'] = third_rule
            test10.loc[i, prompt + '_in_cfs'] = in_cfs
            test10.loc[i, prompt + '_in_data'] = in_dataset
            test10.loc[i, prompt + '_status'] = 0
        except Exception as e:
            test10.loc[i, prompt + '_status'] = 1
            print(e)
test10

one


100%|██████████| 1/1 [00:00<00:00,  4.76it/s]


single positional indexer is out-of-bounds


100%|██████████| 1/1 [00:00<00:00,  5.43it/s]


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,one_label,one_rules,one_rules_followed,one_rule_1,one_rule_2,one_rule_3,one_in_cfs,one_in_data,one_status
0,45,Self-Employed,HS-grad,Married,Blue-Collar,White,Male,50,1.0,3.0,2.0,1.0,1.0,0.0,False,False,0.0
1,46,Private,HS-grad,Divorced,Blue-Collar,White,Male,30,1.0,3.0,2.0,1.0,1.0,0.0,False,False,0.0
2,49,Private,HS-grad,Widowed,Service,White,Male,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,26,Government,Bachelors,Single,Professional,Other,Female,40,0.0,3.0,3.0,1.0,1.0,1.0,False,False,0.0
4,45,Government,Bachelors,Divorced,Service,White,Female,40,1.0,3.0,2.0,1.0,1.0,0.0,False,False,0.0
5,35,Private,Some-college,Married,Blue-Collar,White,Male,60,1.0,3.0,1.0,1.0,0.0,0.0,False,False,0.0
6,29,Private,HS-grad,Single,Blue-Collar,Other,Male,40,1.0,3.0,3.0,1.0,1.0,1.0,False,False,0.0
7,40,Self-Employed,Some-college,Married,Service,White,Male,50,0.0,3.0,1.0,1.0,0.0,0.0,False,False,0.0
8,28,Other/Unknown,HS-grad,Married,Other/Unknown,White,Female,20,1.0,3.0,3.0,1.0,1.0,1.0,True,False,0.0
9,20,Private,Some-college,Single,Service,White,Female,40,1.0,3.0,3.0,1.0,1.0,1.0,False,False,0.0


In [11]:
def get_metrics(df,prompt):
    validity = df[f'{prompt}_label'].mean()
    rules = df[f'{prompt}_rules'].mean()
    rules_ratio = np.mean(df[f'{prompt}_rules_followed']/df[f'{prompt}_rules'])
    first = df[f'{prompt}_rule_1'].mean()
    second = df[f'{prompt}_rule_2'].mean()
    third = df[f'{prompt}_rule_3'].mean()
    return [validity, rules, rules_ratio, first, second, third]
get_metrics(test10,'one')

[0.7777777777777778,
 3.0,
 0.7407407407407407,
 1.0,
 0.7777777777777778,
 0.4444444444444444]

In [16]:
test10 = pd.read_csv('test10.csv')
model_description = """ML-system that predicts wether a person will earn more than 50k $ a year"""
prompts = ['zero']
for prompt in prompts:
    print(prompt)
    exp_m = ExplanationMachine2(model, model_description, string_info(dataset.columns,adult_info), train_dataset, test_dataset,prompt,5)
    exp_m.fit()
    for i in range(test10.shape[0]):
        try:
            example_label, n_rules, rules_followed, first_rule, second_rule,third_rule,in_cfs, in_dataset = exp_m.explain_evaluate(example = test.iloc[[i]], verbose = False)
            os.rename('temp_csv.csv', f'ex_zero_5fcs_{i}.csv')
            os.rename('evaluation.csv', f'eval_zero_5fcs_{i}.csv')
            test10.loc[i, prompt + '_label'] = example_label
            test10.loc[i, prompt + '_rules'] = n_rules
            test10.loc[i, prompt + '_rules_followed'] = rules_followed
            test10.loc[i, prompt + '_rule_1'] = first_rule
            test10.loc[i, prompt + '_rule_2'] = second_rule
            
            test10.loc[i, prompt + '_rule_3'] = third_rule
            test10.loc[i, prompt + '_in_cfs'] = in_cfs
            test10.loc[i, prompt + '_in_data'] = in_dataset
            test10.loc[i, prompt + '_status'] = 0
        except Exception as e:
            test10.loc[i, prompt + '_status'] = 1
test10

zero


100%|██████████| 1/1 [00:00<00:00,  1.91it/s]


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,zero_label,zero_rules,zero_rules_followed,zero_rule_1,zero_rule_2,zero_rule_3,zero_in_cfs,zero_in_data,zero_status
0,45,Self-Employed,HS-grad,Married,Blue-Collar,White,Male,50,1.0,4.0,4.0,1.0,1.0,1.0,False,False,0.0
1,46,Private,HS-grad,Divorced,Blue-Collar,White,Male,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,49,Private,HS-grad,Widowed,Service,White,Male,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,26,Government,Bachelors,Single,Professional,Other,Female,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,45,Government,Bachelors,Divorced,Service,White,Female,40,1.0,4.0,4.0,1.0,1.0,1.0,False,False,0.0
5,35,Private,Some-college,Married,Blue-Collar,White,Male,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6,29,Private,HS-grad,Single,Blue-Collar,Other,Male,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
7,40,Self-Employed,Some-college,Married,Service,White,Male,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8,28,Other/Unknown,HS-grad,Married,Other/Unknown,White,Female,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9,20,Private,Some-college,Single,Service,White,Female,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [17]:
def get_metrics(df,prompt):
    validity = df[f'{prompt}_label'].mean()
    rules = df[f'{prompt}_rules'].mean()
    rules_ratio = np.mean(df[f'{prompt}_rules_followed']/df[f'{prompt}_rules'])
    in_data = df[f'{prompt}_in_data'].mean()
    fail = df[f'{prompt}_status'].mean()
    first = df[f'{prompt}_rule_1'].mean()
    second = df[f'{prompt}_rule_2'].mean()
    third = df[f'{prompt}_rule_3'].mean()
    return [validity, rules, rules_ratio, in_data, first, second, third, fail]
get_metrics(test10,'zero')

[1.0, 4.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.8]

In [18]:
test10 = pd.read_csv('test10.csv')
model_description = """ML-system that predicts wether a person will earn more than 50k $ a year"""
prompts = ['zero']
for prompt in prompts:
    print(prompt)
    exp_m = ExplanationMachine2(model, model_description, string_info(dataset.columns,adult_info), train_dataset, test_dataset,prompt,3)
    exp_m.fit()
    for i in range(test10.shape[0]):
        try:
            example_label, n_rules, rules_followed, first_rule, second_rule,third_rule,in_cfs, in_dataset = exp_m.explain_evaluate(example = test.iloc[[i]], verbose = False)
            os.rename('temp_csv.csv', f'ex_zero_3fcs_{i}.csv')
            os.rename('evaluation.csv', f'eval_zero_3fcs_{i}.csv')
            test10.loc[i, prompt + '_label'] = example_label
            test10.loc[i, prompt + '_rules'] = n_rules
            test10.loc[i, prompt + '_rules_followed'] = rules_followed
            test10.loc[i, prompt + '_rule_1'] = first_rule
            test10.loc[i, prompt + '_rule_2'] = second_rule
            
            test10.loc[i, prompt + '_rule_3'] = third_rule
            test10.loc[i, prompt + '_in_cfs'] = in_cfs
            test10.loc[i, prompt + '_in_data'] = in_dataset
            test10.loc[i, prompt + '_status'] = 0
        except Exception as e:
            test10.loc[i, prompt + '_status'] = 1
test10

zero


100%|██████████| 1/1 [00:00<00:00,  2.15it/s]


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,zero_label,zero_rules,zero_rules_followed,zero_rule_1,zero_rule_2,zero_rule_3,zero_in_cfs,zero_in_data,zero_status
0,45,Self-Employed,HS-grad,Married,Blue-Collar,White,Male,50,1.0,5.0,2.0,1.0,0.0,1.0,False,False,0.0
1,46,Private,HS-grad,Divorced,Blue-Collar,White,Male,30,1.0,3.0,3.0,1.0,1.0,1.0,False,True,0.0
2,49,Private,HS-grad,Widowed,Service,White,Male,40,1.0,4.0,4.0,1.0,1.0,1.0,False,True,0.0
3,26,Government,Bachelors,Single,Professional,Other,Female,40,1.0,4.0,3.0,1.0,1.0,0.0,False,False,0.0
4,45,Government,Bachelors,Divorced,Service,White,Female,40,0.0,4.0,3.0,1.0,1.0,0.0,False,False,0.0
5,35,Private,Some-college,Married,Blue-Collar,White,Male,60,1.0,3.0,3.0,1.0,1.0,1.0,False,False,0.0
6,29,Private,HS-grad,Single,Blue-Collar,Other,Male,40,1.0,3.0,3.0,1.0,1.0,1.0,False,False,0.0
7,40,Self-Employed,Some-college,Married,Service,White,Male,50,1.0,4.0,3.0,1.0,1.0,1.0,False,False,0.0
8,28,Other/Unknown,HS-grad,Married,Other/Unknown,White,Female,20,1.0,4.0,4.0,1.0,1.0,1.0,False,True,0.0
9,20,Private,Some-college,Single,Service,White,Female,40,1.0,3.0,2.0,1.0,1.0,0.0,False,False,0.0


In [19]:
get_metrics(test10,'zero')

[0.9, 3.7, 0.8316666666666667, 0.3, 1.0, 0.9, 0.7, 0.0]

In [12]:
test10 = pd.read_csv('test10.csv')
model_description = """ML-system that predicts wether a person will earn more than 50k $ a year"""
prompts = ['zero']
for prompt in prompts:
    print(prompt)
    exp_m = ExplanationMachine2(model, model_description, string_info(dataset.columns,adult_info), train_dataset, test_dataset,prompt,1)
    exp_m.fit()
    for i in range(test10.shape[0]):
        try:
            example_label, n_rules, rules_followed, first_rule, second_rule,third_rule,in_cfs, in_dataset = exp_m.explain_evaluate(example = test.iloc[[i]], verbose = False)
            os.rename('temp_csv.csv', f'ex_zero_1fcs_{i}.csv')
            os.rename('evaluation.csv', f'eval_zero_1fcs_{i}.csv')
            test10.loc[i, prompt + '_label'] = example_label
            test10.loc[i, prompt + '_rules'] = n_rules
            test10.loc[i, prompt + '_rules_followed'] = rules_followed
            test10.loc[i, prompt + '_rule_1'] = first_rule
            test10.loc[i, prompt + '_rule_2'] = second_rule
            
            test10.loc[i, prompt + '_rule_3'] = third_rule
            test10.loc[i, prompt + '_in_cfs'] = in_cfs
            test10.loc[i, prompt + '_in_data'] = in_dataset
            test10.loc[i, prompt + '_status'] = 0
        except Exception as e:
            test10.loc[i, prompt + '_status'] = 1
test10

zero


100%|██████████| 1/1 [00:00<00:00,  2.51it/s]


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,zero_status,zero_label,zero_rules,zero_rules_followed,zero_rule_1,zero_rule_2,zero_rule_3,zero_in_cfs,zero_in_data
0,45,Self-Employed,HS-grad,Married,Blue-Collar,White,Male,50,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,46,Private,HS-grad,Divorced,Blue-Collar,White,Male,30,0.0,1.0,7.0,3.0,1.0,0.0,1.0,False,True
2,49,Private,HS-grad,Widowed,Service,White,Male,40,0.0,1.0,4.0,2.0,1.0,1.0,0.0,True,False
3,26,Government,Bachelors,Single,Professional,Other,Female,40,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,45,Government,Bachelors,Divorced,Service,White,Female,40,0.0,1.0,4.0,4.0,1.0,1.0,1.0,True,False
5,35,Private,Some-college,Married,Blue-Collar,White,Male,60,0.0,1.0,4.0,2.0,0.0,1.0,1.0,False,True
6,29,Private,HS-grad,Single,Blue-Collar,Other,Male,40,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,40,Self-Employed,Some-college,Married,Service,White,Male,50,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,28,Other/Unknown,HS-grad,Married,Other/Unknown,White,Female,20,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,20,Private,Some-college,Single,Service,White,Female,40,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
get_metrics(test10,'zero')

[1.0, 4.75, 0.6071428571428572, 0.5, 0.75, 0.75, 0.75, 0.6]

In [8]:
test10 = pd.read_csv('test10.csv')
model_description = """ML-system that predicts wether a person will earn more than 50k $ a year"""
prompts = ['one']
for prompt in prompts:
    print(prompt)
    exp_m = ToTExplanationMachine(model, model_description, string_info(dataset.columns,adult_info), train_dataset, test_dataset,prompt,5, 3 )
    exp_m.fit()
    for i in range(test10.shape[0]):
        try:
            example_label, n_rules, rules_followed, first_rule, second_rule, third_rule, in_dataset = exp_m.explain_evaluate(user_data = test.iloc[[i]], verbose = False)
            os.rename('temp_csv.csv', f'ex_5fcs_{i}.csv')
            os.rename('evaluation.csv', f'eval_5fcs_{i}.csv')            
            test10.loc[i, prompt + '_label'] = example_label
            test10.loc[i, prompt + '_rules'] = n_rules
            test10.loc[i, prompt + '_rules_followed'] = rules_followed
            test10.loc[i, prompt + '_rule_1'] = first_rule
            test10.loc[i, prompt + '_rule_2'] = second_rule
            test10.loc[i, prompt + '_rule_3'] = third_rule
            #test10.loc[i, prompt + '_in_cfs'] = in_cfs
            test10.loc[i, prompt + '_in_data'] = in_dataset
            test10.loc[i, prompt + '_status'] = 0
            break
        except Exception as e:
            test10.loc[i, prompt + '_status'] = 1
            print(e)
test10


one


100%|██████████| 1/1 [00:00<00:00,  4.59it/s]


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,one_label,one_rules,one_rules_followed,one_rule_1,one_rule_2,one_rule_3,one_in_data,one_status
0,45,Self-Employed,HS-grad,Married,Blue-Collar,White,Male,50,1.0,3.0,2.0,1.0,1.0,0.0,False,0.0
1,46,Private,HS-grad,Divorced,Blue-Collar,White,Male,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,49,Private,HS-grad,Widowed,Service,White,Male,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,26,Government,Bachelors,Single,Professional,Other,Female,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,45,Government,Bachelors,Divorced,Service,White,Female,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,35,Private,Some-college,Married,Blue-Collar,White,Male,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,29,Private,HS-grad,Single,Blue-Collar,Other,Male,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,40,Self-Employed,Some-college,Married,Service,White,Male,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,28,Other/Unknown,HS-grad,Married,Other/Unknown,White,Female,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,20,Private,Some-college,Single,Service,White,Female,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
